In [ ]:
import torch
import torchvision
import torch.nn as nn 
import numpy as np
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt

#디버깅 모듈 설치
import pdb




In [ ]:
#우리가 사용할 컴퓨터를 check하는 부분, cpu/gpu 지원받을지..
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 하이퍼파라미터 설정

input_size = 784 # mnist는 28x28x1로 구성
hidden_size = 500 # 은닉층의 unit 수
num_classes = 10 # 카테고리 개수
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# dataset 로딩 - 2개의 과정을 거친다
# train_dataset, test_dataset 데이터 로딩 1단계
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train = True, #traindataset만 저장
                                           transform = transforms.ToTensor(),
                                           download = True) #traindataset을 스케일링하여 다운로드하겠다.
test_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train = False, #test_dataset만 저장, test옵션은 따로 없음.
                                           transform = transforms.ToTensor())

# 데이터 로딩 2단계.. BatchSize 이용
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle =True)

# 100개씩 쪼개서 저장..
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle =False) #테스트는 학습이아니므로 shuffle 필요 없다.

## Convolution NeuralNet Model 생성

In [ ]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    self.layer1 =nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=2),
        nn.ReLU())
    
    self.layer2 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.layer3 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.ReLU())
    
    self.layer4 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.layer5 = nn.Linear(7*7*32, num_classes)

  def forward(self, x):

    # pdb.set_trace() # 2.디버깅
        # n : 다음 라인 가르킨다.
        # x.size, out.size  등 크기 확인 필요

     out = self.layer1(x)
     out = self.layer2(out)
     out = self.layer3(out)
     out = self.layer4(out)
     out = out.reshape(out.size(0),-1)
     out = self.layer5(out)

     return out



In [ ]:
model = ConvNet(num_classes).to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # pdb.set_trace() # 1. 디버깅.. 주석으로 막아두고 후에 사용.. outputs에서 호출되며 실행 
         
        # l : pdb로 어느부분을 디버깅 하려 하는지 알 수 있음.. 호출되는 함수인 forward가 나타남.
        # images.size(), images.shape : 이미지 사이즈 알아보기([batch_size, channel, height, width])
        # labels.size() : 라벨의 사이즈 알 수 있다.
        # labels : 라벨을 전부 뽑는다.
        # quit, q : 빠져나온다. 빠져나오면서 error 나오지만 신경 안써도됨.
        

        

        outputs = model(images) # =model.forward(images)...모델에 입력값을 넣어 예측값을 반환. 이때 forward함수 호출.
        
        loss = loss_function

        # pdb.set_trace() #3.디버깅
            # loss값 확인 가능. loss.item()..
        n(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 100 ==0:
            print('Epoch[{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
model.eval() # 테스트모델이다..라는 의미
with torch.no_grad(): # 실제로 학습할 필요가 없으면 no.grad()구문을 넣어준다... no gradiation.. 편미분 안하겠다.
    correct = 0
    total = 0
    for images, labels in test_loader: # 인덱스를 굳이 안받아도 되니깐 enumeratorr가 빠졌다.
        images = images.to(device) #한줄로 죽 펼친 다음에 입력받아서
        labels = labels.to(device) # 라벨도 마찬가지로
        outputs = model(images) #모델에 넣고
        _, predicted = torch.max(outputs.data, 1)
        # total은 푼 문제, correct는 맞춘 문제수..이런식으로 카운팅이 들어간다
     
        total += labels.size(0) 
        correct += (predicted == labels).sum().item()
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
images.size()
images[0,0].size()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

idx = 12
image = np.array(images[idx,0].detach().cpu()) #detach().. 연산에서 분리. gpu에서 내려줘야하므로 .cpu() 사용
label = labels[idx].item()
label
pred = predicted[idx].item()
pred

plt.imshow(image * 255) # tensor로 scale 되어있어서 원래 이미지 픽셀값으로 되돌린다.
print('Label : ', label)
print('Predict : ', predicted)

In [ ]:
for idx in range(30):
    image = np.array(images[idx,0].detach().cpu())
    label = labels[idx].item()
    pred = predicted[idx].item()
    plt.figure()
    plt.title(f'Label:{label} Pred:{pred}')
    plt.imshow(image * 255)
